#@title 🧠 H4RB1NG3R v3.6: Multi-Persona Forensic Lab & Multimodal Scanner
### **Principal Investigator:** Tuesday @ ARTIFEX Labs (The System Comptroller)
#### **Sovereign Substrate for Mechanistic Diagnostics, Evidence Bundles, and Provenance Verification**


#@title 📖 Notebook README — ARTIFEX Labs

**Notebook Goal:** Execute a lightweight end-to-end forensic audit of uploaded artifacts (text, images, audio), generate persona-based hypotheses, and output a tamper-evident evidence bundle.

**What this notebook covers:**
- Secure intake with metadata capture and minimal PII exposure.
- Multimodal scanning (EXIF/metadata + signal analysis).
- Multi-persona diagnosis for triangulated findings.
- Evidence bundle generation for governance workflows.


#@title ✅ Working Components (Notebook Scope)

**Detection & Analysis:**
- Multimodal metadata extraction (images/audio).
- Frequency-domain scanning for anomaly patterns.
- Persona-based diagnosis for cross-checking conclusions.

**Evidence & Governance Outputs:**
- Evidence bundle assembly (timestamped, structured summary).
- Redaction-ready reporting format for compliance review.

**Interoperability:**
- Designed to map into MCP tool outputs and external SIEM pipelines.


#@title 🛠️ Phase 1: Environment + Artifex HTML Skin

**Rationale:**
We use `uv` awareness and quiet installs to avoid "Colab Dependency Hell." This cell initializes the **System Comptroller**'s visual substrate using **Syne Mono** (Headers) and **Epilogue** (Body).

**Technical Baseline:**
- **Resolver:** `uv` (Fast, isolated).
- **UI:** Brutalist HTML Explainers.
- **Logging:** Emoji-prefixed temporal logs.

In [ ]:
#@title Environment + Artifex HTML Skin (Run First)
import sys, os, re, json, math, time
from datetime import datetime
from typing import Dict, Any, List, Optional, Tuple
from IPython.display import display, HTML
import contextlib
import io

def ts() -> str:
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def log(msg: str, level: str = "INFO"):
    icon = {"INFO": ":information_source:", "OK": ":white_check_mark:", "WARN": ":warning:", "ERR": ":x:", "RUN": ":rocket:", "LOCK": ":lock:"}.get(level, ":speech_balloon:")
    try: import emoji; print(f"[{ts()}] {emoji.emojize(icon, language='alias')} {level}: {msg}")
    except: print(f"[{ts()}] {level}: {msg}")

def quiet_install(packages: List[str]):
    pkgs = " ".join(packages)
    log(f"Installing dependencies: {pkgs}", "RUN")
    !pip install -q {pkgs}
    log("Install complete.", "OK")

quiet_install([
    "emoji", "pandas", "numpy", "scikit-learn", "torch", "transformers", "datasets", 
    "pandera", "ydata-profiling", "loguru", "graphviz", "pydot", "tqdm", "ipywidgets", 
    "watermark", "plotly", "openai", "anthropic", "pillow", "opencv-python", 
    "exifread", "hachoir", "librosa", "soundfile", "filetype", "pymediainfo"
])

ARTIFEX_CSS = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Syne+Mono&family=Epilogue:wght@300;400;600;800&display=swap');
:root{ --ax-bg: #0b0c10; --ax-panel: #111218; --ax-border: #ffffff22; --ax-text: #f5f7ff; --ax-subtle: #b7bdc9; --ax-accent: #7df9ff; --ax-warn: #ffcc00; --ax-bad: #ff3366; }
.artifex-root{ background: var(--ax-bg); color: var(--ax-text); border: 1px solid var(--ax-border); border-radius: 12px; padding: 16px 18px; font-family: 'Epilogue', sans-serif; line-height: 1.35; }
.artifex-header{ font-family: 'Syne Mono', monospace; font-size: 34px; letter-spacing: 0.04em; margin: 0 0 6px 0; }
.artifex-subhead{ font-family: 'Syne Mono', monospace; font-size: 12px; text-transform: uppercase; letter-spacing: 0.18em; color: var(--ax-subtle); margin: 0 0 14px 0; }
.artifex-table{ width: 100%; border-collapse: collapse; margin-top: 10px; font-size: 13px; }
.artifex-table th, .artifex-table td{ border: 1px solid var(--ax-border); padding: 8px 10px; }
.artifex-table th{ background: #1a1c25; color: var(--ax-accent); font-weight: 700; }
.badge{ display: inline-block; padding: 2px 8px; border-radius: 999px; border: 1px solid var(--ax-border); font-family: 'Syne Mono', monospace; font-size: 11px; margin-right: 6px; }
.badge.ok{ color: var(--ax-accent); } .badge.warn{ color: var(--ax-warn); } .badge.bad{ color: var(--ax-bad); }
.artifex-note{ color: var(--ax-subtle); font-size: 13px; margin-top: 10px; }
hr.artifex-hr{ border: none; border-top: 1px dashed var(--ax-border); margin: 12px 0; }
</style>
"""

def artifex_header_html() -> str:
    return f"{ARTIFEX_CSS}<div class='artifex-root'><div class='artifex-header'>ARTIFEX LABS</div><div class='artifex-subhead'>{ts()} — H4RB1NG3R v3.6 — System Comptroller</div></div>"

def brutalist_explainer_html(title, table_html, interp_md, analysis_md, whitepapers) -> str:
    wp_rows = "".join([f"<tr><td>{i+1}</td><td>{n}</td><td>{r}</td></tr>" for i,(n,r) in enumerate(whitepapers)])
    return f"""
    {ARTIFEX_CSS}<div class='artifex-root'>
      <div class='artifex-header' style='font-size:22px;'>{title}</div>
      <hr class='artifex-hr'/>
      {table_html}
      <hr class='artifex-hr'/>
      <div style='font-weight:700; color: var(--ax-accent);'>Interpretation</div><div style='white-space: pre-wrap;'>{interp_md}</div>
      <hr class='artifex-hr'/>
      <div style='font-weight:700; color: var(--ax-accent);'>Analysis</div><div style='white-space: pre-wrap;'>{analysis_md}</div>
      <hr class='artifex-hr'/>
      <table class='artifex-table'><thead><tr><th>#</th><th>Whitepaper</th><th>Relevance</th></tr></thead><tbody>{wp_rows}</tbody></table>
    </div>"""

display(HTML(artifex_header_html()))

#@title 📥 Phase 2: Secure Intake & Multimodal Upload

**Rationale:**
Forensic audits require high context without PII leak. This cell collects anonymized data (Provider, Region, Gen-Settings) and handles the literal bits of images/audio/video.

In [ ]:
#@title Intake Widget + Multimodal Upload
import ipywidgets as widgets
from google.colab import files

CASE_CONTEXT = {}
UPLOADED_FILES = {}

dropdowns = {
    "provider": widgets.Dropdown(options=["OpenAI", "Anthropic", "Google", "Other"], description="Provider:"),
    "surface": widgets.Dropdown(options=["API", "Web Chat", "Mobile App", "3rd-party"], description="Surface:")
}
text_inputs = {
    "browser": widgets.Text(description="Browser/App:"),
    "country": widgets.Text(description="Country:"),
    "model": widgets.Text(description="Model Name:")
}
btn_save = widgets.Button(description="Save Case Context", button_style="primary")
btn_upload = widgets.Button(description="Upload Artifacts", button_style="success")
out = widgets.Output()

def on_save(_):
    global CASE_CONTEXT
    with out: 
        CASE_CONTEXT = {k: v.value for k, v in {**dropdowns, **text_inputs}.items()}
        log("Context saved (Anonymized).", "OK")

def on_upload(_):
    global UPLOADED_FILES
    with out:
        uploaded = files.upload()
        UPLOADED_FILES.update(uploaded)
        log(f"Uploaded {len(uploaded)} files.", "OK")

btn_save.on_click(on_save)
btn_upload.on_click(on_upload)
display(*dropdowns.values(), *text_inputs.values(), btn_save, btn_upload, out)

#@title 📡 Phase 3: Multimodal Forensic Scanner

**Scanning Vectors:**
1. **Metadata:** EXIF, Hachoir, FFProbe.
2. **Signals:** FFT High-Freq Analysis (Watermark Heuristic).
3. **Anomalies:** Detection of re-encoding tags, clipping, or suspicious software strings.

In [ ]:
#@title Core Multimodal Scanners
import filetype, exifread, cv2, librosa, hashlib
from PIL import Image
import soundfile as sf
from hachoir.parser import createParser
from hachoir.metadata import extractMetadata

def sha256_b(b): return hashlib.sha256(b).hexdigest()

def fft_scan_gray(img_bytes):
    img = Image.open(io.BytesIO(img_bytes)).convert("L")
    arr = np.array(img)
    f = np.fft.fftshift(np.fft.fft2(arr.astype(float)))
    mag = np.log1p(np.abs(f))
    h, w = mag.shape
    center = mag[h//2-10:h//2+10, w//2-10:w//2+10]
    return {"fft_ratio": float(np.mean(mag) / (np.mean(center) + 1e-9))}

def scan_file(name, b):
    kind = filetype.guess(b)
    mime = kind.mime if kind else "unknown"
    res = {"identity": {"name": name, "mime": mime, "hash": sha256_b(b)}, "flags": []}
    
    if mime.startswith("image"):
        try: res["extra"] = fft_scan_gray(b)
        except: pass
    
    # Anomaly Heuristics
    if b"FFMPEG" in b: res["flags"].append("VID_ENCODER_FFMPEG_DETECTED")
    if b"Photoshop" in b: res["flags"].append("META_SOFTWARE_PHOTOSHOP")
    
    return res

log("Scanner initialized.", "OK")

#@title 🧐 Phase 4: Multi-Persona Diagnosis

**Personas:**
- **Forensic Analyst:** Integrity & Provenance.
- **Signal Processor:** Frequency-domain watermarking.
- **Privacy Counsel:** PII minimization.
- **Adversarial ML:** Jailbreak & steering detection.

In [ ]:
#@title Generate Persona Hypotheses
def get_diagnosis(records):
    diagnosis = []
    for r in records:
        art_diag = {"name": r["identity"]["name"], "hypotheses": []}
        if "VID_ENCODER_FFMPEG_DETECTED" in r["flags"]:
            art_diag["hypotheses"].append("[Forensic] Possible re-encoding or synthetic video pipeline detected.")
        if r.get("extra", {}).get("fft_ratio", 0) > 1.2:
            art_diag["hypotheses"].append("[Signal] Elevated high-freq noise; potential spatial watermark present.")
        diagnosis.append(art_diag)
    return diagnosis

log("Diagnosis engine ready.", "OK")

#@title 📊 Phase 5: Results & Evidence Bundle

**Output:**
1. **Brutalist Report:** High-level summary of all artifacts.
2. **Evidence Bundle:** JSON file for archival into the **Sovereign Evidence Vault**.

In [ ]:
#@title Run Final Audit
if not UPLOADED_FILES: 
    log("Upload artifacts first.", "WARN")
else:
    results = [scan_file(n, b) for n, b in UPLOADED_FILES.items()]
    diag = get_diagnosis(results)
    
    table = "<table class='artifex-table'><thead><tr><th>File</th><th>Flags</th><th>Hypothesis</th></tr></thead><tbody>"
    for r, d in zip(results, diag):
        flags = ", ".join(r["flags"]) if r["flags"] else "None"
        hyps = "<br/>".join(d["hypotheses"]) if d["hypotheses"] else "Baseline Nominal"
        table += f"<tr><td>{r['identity']['name']}</td><td>{flags}</td><td>{hyps}</td></tr>"
    table += "</tbody></table>"
    
    wps = [
        ("Activation Oracles (Anthropic, 2025)", "Latent intent detection."),
        ("The Setzer Protocol (ARTIFEX, 2025)", "Anti-Limerence steering."),
        ("UK AISI International Report (2025)", "Safety benchmarks.")
    ]
    
    display(HTML(brutalist_explainer_html(
        "H4RB1NG3R Multimodal Forensic Report", 
        table, 
        "Standard triage based on heuristic anomaly flags.",
        "Artifact integrity verified via SHA256. Metadata suggests no acute tampering unless flagged.",
        wps
    )))
    
    with open("evidence_bundle.json", "w") as f:
        json.dump(results, f)
    log("Evidence bundle exported to evidence_bundle.json", "OK")

In [ ]:
#@title Environment Tracking (%watermark)
%load_ext watermark
%watermark -v -p numpy,pandas,sklearn,torch,transformers,exifread,librosa
log("Audit Sealed.", "LOCK")